In [1]:
import pandas as pd

In [2]:
path ="../data/sample_superstore.csv"
df = pd.read_csv(path, encoding='latin1')

In [3]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
df.columns.tolist()

['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

In [5]:
neccessary_columns = [
    'Order ID', 'Order Date', 'Customer ID',  'Sales'
]

df_filtered = df[neccessary_columns].copy()

print(df_filtered.shape)
df_filtered.describe(include='all').round(2)

(9994, 4)


,Order ID,Order Date,Customer ID,Sales
count,9994,9994,9994,9994.00
unique,5009,1237,793,NaN
top,CA-2017-100111,9/5/2016,WB-21850,NaN
freq,14,38,37,NaN
mean,NaN,NaN,NaN,229.86
std,NaN,NaN,NaN,623.25
min,NaN,NaN,NaN,0.44
25%,NaN,NaN,NaN,17.28
50%,NaN,NaN,NaN,54.49
75%,NaN,NaN,NaN,209.94


In [6]:
column_mapping = {
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Customer ID': 'customer_id',
    'Sales': 'sales_amount'
}

df_filtered.rename(columns=column_mapping, inplace=True)

df_filtered = df_filtered.groupby(['order_id', 'order_date', 'customer_id'])\
    .agg({'sales_amount': 'sum'})\
        .reset_index()

Data Type checks

In [7]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      5009 non-null   object 
 1   order_date    5009 non-null   object 
 2   customer_id   5009 non-null   object 
 3   sales_amount  5009 non-null   float64
dtypes: float64(1), object(3)
memory usage: 156.7+ KB


In [8]:
# Data Type Fixes
df_filtered['order_date'] = pd.to_datetime(df_filtered['order_date'])
df_filtered['order_id'] = df_filtered['order_id'].str.strip().astype('string')
df_filtered['customer_id'] = df_filtered['customer_id'].str.strip().astype('string')
df_filtered['sales_amount'] = pd.to_numeric(df_filtered['sales_amount'], errors='coerce')

df_filtered.dtypes

order_id        string[python]
order_date      datetime64[ns]
customer_id     string[python]
sales_amount           float64
dtype: object

Sanity Check of Data

date

In [9]:
df_filtered['order_date'].describe()

count                             5009
mean     2016-05-07 16:04:47.482531328
min                2014-01-03 00:00:00
25%                2015-05-31 00:00:00
50%                2016-07-10 00:00:00
75%                2017-05-19 00:00:00
max                2017-12-30 00:00:00
Name: order_date, dtype: object

In [10]:
# time = pd.DataFrame()
# time['date'] = pd.date_range(start=df_filtered['order_date'].min(), end=df_filtered['order_date'].max(), freq='D')
# time.describe()

sales amount

In [11]:
df_filtered['sales_amount'].min(), df_filtered['sales_amount'].max()

df_filtered['sales_amount'].describe()

count     5009.000000
mean       458.614666
std        954.730953
min          0.556000
25%         37.630000
50%        151.960000
75%        512.064000
max      23661.228000
Name: sales_amount, dtype: float64

order id

In [12]:
(df_filtered['order_id'].value_counts().sort_values(ascending=False) > 1).sum()

np.int64(0)

customer id

In [13]:
df_filtered['customer_id'].value_counts().head()

customer_id
EP-13915    17
CK-12205    13
SH-19975    13
JE-15745    13
NS-18640    13
Name: count, dtype: Int64

In [14]:
df_filtered.head()
df_filtered.to_excel("../data/cleaned_sample_superstore.xlsx", index=False)